In [2]:
#Imports
import torch
import matplotlib.pyplot as plt
from PIL import Image
import utils

from ultralytics import YOLO
from ultralytics.models.yolo.segment.train import SegmentationTrainer


/Users/thomas/Documents/VBTI/Python/ultralytics/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Get dataloader & model
st = SegmentationTrainer(cfg="args.yaml", overrides=dict(data="dataset.yaml", device="cpu"))
# st._setup_train(10)

dataloader = st.get_dataloader("/Users/thomas/Documents/VBTI/DataAnalysis/autodl/dataset/yolo-converted/cucumber_dataset/train/images", batch_size=1)

model = YOLO("/Users/thomas/Documents/School/TU:e/1. Master/Year 3/Graduation/Preparation Phase/Data Analysis/models/vdl-smart-trim-s-0-models-20230726-100018__yolo8m_imgsize_960/weights/best.pt")

Ultralytics YOLOv8.0.188 🚀 Python-3.9.6 torch-2.0.1 CPU (Apple M1 Pro)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.yaml, data=dataset.yaml, epochs=250, patience=50, batch=8, imgsz=960, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=vdl-smart-trim, name=yolo_vdl_investigation, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=33, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=F

In [10]:
st.train()


AttributeError: 
            'IterableSimpleNamespace' object has no attribute 'freeze'. This may be caused by a modified or out of date ultralytics
            'default.yaml' file.
Please update your code with 'pip install -U ultralytics' and if necessary replace
            /Users/thomas/Documents/VBTI/Python/ultralytics/ultralytics/cfg/default.yaml with the latest version from
            https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/default.yaml
            

In [3]:
# Plot 5 images with predictions
for i in range(5):
    # Get a batch of training data
    data = next(iter(dataloader))

    inputs = data['im_file'][0]

    # Forward pass the data through the model
    # Perform object detection on an image using the model
    results = model(inputs)

    images = []
    # Show the results
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.show()  # show image
        # im.save('results.jpg')  # save image
        # images.append(im)


image 1/1 /Users/thomas/Documents/VBTI/DataAnalysis/autodl/dataset/yolo-converted/cucumber_dataset/train/images/im66.jpeg: 544x960 16 Cucumbers, 7 Cucumber Stems, 33 Leaf Stems, 28 Main Stems, 13 Nodes, 9 Plastic Rings, 390.8ms
Speed: 4.3ms preprocess, 390.8ms inference, 31.4ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /Users/thomas/Documents/VBTI/DataAnalysis/autodl/dataset/yolo-converted/cucumber_dataset/train/images/im37.jpeg: 544x960 6 Cucumbers, 4 Cucumber Stems, 14 Leaf Stems, 17 Main Stems, 9 Nodes, 8 Plastic Rings, 552.6ms
Speed: 18.0ms preprocess, 552.6ms inference, 22.8ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /Users/thomas/Documents/VBTI/DataAnalysis/autodl/dataset/yolo-converted/cucumber_dataset/train/images/im8.jpeg: 544x960 10 Cucumbers, 4 Cucumber Stems, 23 Leaf Stems, 19 Main Stems, 10 Nodes, 7 Plastic Rings, 437.5ms
Speed: 9.7ms preprocess, 437.5ms inference, 24.6ms postprocess per image at shape (1, 3, 544, 960)

image 1/1 /Users/

In [5]:
# Setup 
epsilons = [0, .05, .1, .15, .2, .25, .3]
# epsilons = [0, .1, .3]

torch.manual_seed(42)

# Define what device we are using
print("mps Available: ",torch.has_mps)
device = torch.device("cpu")

mps Available:  True


In [25]:
def test(model, device, test_loader, epsilon, batch_size):
    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for datadict in test_loader:
        data = datadict['im_file'][0]
        masks = datadict['masks']
        cls = datadict['cls']
        bboxes = datadict['bboxes']
        
        # print(masks, cls, bboxes)

        # Send the data and label to the device
        # data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        # data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output  # get the index of the max log-probability

        # If the initial prediction is wrong, don't bother attacking, just move on
        # if init_pred != target:
        #     continue

        # Calculate the loss
        # loss = 

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect ``datagrad``
        data_grad = data.grad.data

        # Restore the data to its original scale
        # data_denorm = denorm(data_grad)

        # Call FGSM Attack
        perturbed_data = utils.fgsm_attack(data, epsilon, data_grad)

        # Reapply normalization
        # perturbed_data_normalized = transforms.Normalize((0.1307,), (0.3081,))(perturbed_data)
        perturbed_data_normalized = perturbed_data

        # Re-classify the perturbed image
        output = model(perturbed_data_normalized)
        # 
        # # Check for success
        # final_pred = torch.max(output, 1)  # get the index of the max log-probability
        # for k in range(batch_size):
        #     fp = final_pred.indices
        #     ip = init_pred.indices
        #     tp = target
        # 
        #     if fp == tp:
        #         correct += 1
        #         # Special case for saving 0 epsilon examples
        #         if epsilon == 0 and len(adv_examples) < 8:
        #             adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        #             adv_examples.append((ip, fp, adv_ex))
        #     else:
        #         # Save some adv examples for visualization later
        #         if len(adv_examples) < 8:
        #             adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        #             adv_examples.append((ip, fp, adv_ex))

    # Calculate final accuracy for this epsilon
    final_acc = correct / float(len(test_loader))
    print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(test_loader)} = {final_acc}")
    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [ ]:
# Execute
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, device, dataloader, eps, device)
    accuracies.append(acc)
    examples.append(ex)


image 1/1 /Users/thomas/Documents/VBTI/DataAnalysis/autodl/dataset/yolo-converted/cucumber_dataset/train/images/im73.jpeg: 544x960 7 Cucumbers, 7 Cucumber Stems, 27 Leaf Stems, 26 Main Stems, 12 Nodes, 9 Plastic Rings, 397.4ms
Speed: 4.2ms preprocess, 397.4ms inference, 55.3ms postprocess per image at shape (1, 3, 544, 960)


In [ ]:
import numpy as np

# Prepare plot
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
from numpy import argmax

# Plot several examples of adversarial samples at each epsilon
cnt = 0
plt.figure(figsize=(12,16))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[i]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel(f"Eps: {epsilons[i]}", fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title(f"MaP:?, MaP:?", fontdict={"fontsize": 8})
        plt.imshow(ex.transpose(1, 2, 0))
plt.tight_layout()
plt.show()